In [2]:
from twnews.dataset.storage import NewsStorage
news_storage = NewsStorage()
news_texts = news_storage.get_texts()
print news_texts[0]


2016-03-23 00:43:11.076892: Start of loading news from /home/avybornov/twnews_data/rss
2016-03-23 00:43:11.268799: News successfully loaded
Политолог: Россия стремится к миру в Сирии, сокращая группировку войск


In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
tvf = TfidfVectorizer()

In [9]:
tfidf_matrix = tvf.fit_transform(news_texts)

In [17]:
X = tfidf_matrix.transpose()

In [19]:
corpus = tvf.get_feature_names()

In [21]:
print len(corpus)
s = 10000
print corpus[s:s+10]

29200
[u'\u043a\u043e\u0432\u0442\u0443\u043d', u'\u043a\u043e\u0433\u0430\u043b\u044b\u043c\u0430\u0432\u0438\u0430', u'\u043a\u043e\u0433\u0430\u043d', u'\u043a\u043e\u0433\u0434\u0430', u'\u043a\u043e\u0433\u043e', u'\u043a\u043e\u0434', u'\u043a\u043e\u0434\u0430', u'\u043a\u043e\u0434\u0435\u043a\u0441', u'\u043a\u043e\u0434\u0435\u043a\u0441\u0430', u'\u043a\u043e\u0434\u0435\u043a\u0441\u0435']


In [52]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

wordnet_lemmatizer = WordNetLemmatizer()

#https://pythonprogramming.net/stop-words-nltk-tutorial/?completed=/tokenizing-words-sentences-nltk-tutorial/

stop_words = set(stopwords.words('russian'))

text = news_texts[0]
word_tokens = [wordnet_lemmatizer.lemmatize(w.decode('utf-8')) for w in word_tokenize(text)]
filtered_sentence = [w for w in word_tokens if not w in stop_words]

print text
#print word_tokens
#print (map(unicode, word_tokens))
print '-------------------'
for x in word_tokens: print x
print '-------------------'
for x in filtered_sentence: print x

Политолог: Россия стремится к миру в Сирии, сокращая группировку войск
-------------------
Политолог
:
Россия
стремится
к
миру
в
Сирии
,
сокращая
группировку
войск
-------------------
Политолог
:
Россия
стремится
миру
Сирии
,
сокращая
группировку
войск


In [50]:
nltk.download()


showing info http://www.nltk.org/nltk_data/


True